# Simple RAG System

## Set up API access

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

In [3]:
# Load Groq API key from .env
load_dotenv()

from langchain.document_loaders import PyPDFLoader

ModuleNotFoundError: No module named 'langchain'

## Load Documents into Vector Store

## Set up Retriever

## Set up LLM call